In [52]:
import os
import pandas as pd

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
print(torch.cuda.get_device_name(0))

Quadro RTX 6000


In [81]:
train_file_path='../../dataset/UCF/test/'
df_UCF_train=pd.concat([pd.read_pickle(train_file_path+file, compression='gzip') for file in os.listdir(train_file_path)], ignore_index=True)
df_UCF_train['input']= df_UCF_train['TAXPAYER_NAME']+' '+df_UCF_train['mission_spellchk']+' '+df_UCF_train['prgrm_dsc_spellchk']

In [82]:
# Create sentence and label lists
sentences = df_UCF_train.input.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = preprocessing.LabelEncoder().fit_transform(df_UCF_train.NTEE1.values)

In [83]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'green', '##croft', 'go', '##sh', '##en', 'inc', 'green', '##croft', 'go', '##sden', 'provides', 'active', ',', 'affordable', 'retirement', 'living', 'with', 'supportive', 'services', ',', 'assisted', 'living', ',', 'and', 'skilled', 'nursing', 'care', '.', 'charity', 'care', 'for', 'eye', 'june', '30', ',', '2014', 'amounted', 'to', '$', '4', '##e', '##27', '##9', '##39', '##22', '.', 'nursing', 'services', '-', 'green', '##croft', 'go', '##sden', 'is', 'health', 'facilities', 'provide', 'state', '-', 'of', '-', 'the', '-', 'art', 'health', 'care', 'techniques', 'and', 'technology', 'in', 'an', 'inviting', 'and', 'supportive', 'residential', 'environment', '.', 'our', 'setting', 'is', 'appropriate', 'for', 'individuals', 'who', 'are', 'recovering', 'from', 'surgery', 'or', 'illness', ',', 'need', 'rehabilitation', ',', 'or', 'require', 'ongoing', 'long', '-', 'term', 'nursing', 'care', '.', 'our', 'rest', '##ora', '##tive', 'care', 'philosophy', 

In [ ]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
# MAX_LEN = np.max([len(tokens) for tokens in tokenized_texts])
MAX_LEN = 128

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [89]:
print(tokenized_texts[1], len(tokenized_texts))

['[CLS]', 'fairbanks', 'concert', 'association', 'to', 'present', ',', 'promote', ',', 'and', 'sponsor', 'artistic', '##ally', 'and', 'culturally', 'diverse', 'performing', 'arts', 'events', 'and', 'educational', 'opportunities', 'of', 'the', 'highest', 'quality', 'from', 'stages', 'around', 'the', 'world', '.', 'the', 'fairbanks', 'concert', 'association', 'proudly', 'presented', '9', 'guest', 'artists', ',', '10', 'concerts', ',', '4', 'school', 'performances', ',', '2', 'workshops', ',', '5', 'rural', 'outreach', 'performances', ',', 'reside', '##ncies', 'at', 'west', 'valley', 'high', 'school', 'and', 'the', 'fairbanks', 'youth', 'facility', ',', 'performances', 'for', 'the', 'pioneer', 'home', 'and', 'the', 'den', '##ali', 'center', ',', '5', 'performances', 'on', 'ku', '##an', 'is', 'alaska', 'live', '-', 'in', 'all', 'more', 'than', '30', 'events', 'this', 'season', '.', 'it', 'was', 'another', 'wonderful', 'season', 'from', 'an', 'artistic', 'stand', '##point', 'and', 'very', '

In [90]:
input_ids[1]

array([  101, 24502,  4164,  2523,  2000,  2556,  1010,  5326,  1010,
        1998, 10460,  6018,  3973,  1998, 20547,  7578,  4488,  2840,
        2824,  1998,  4547,  6695,  1997,  1996,  3284,  3737,  2013,
        5711,  2105,  1996,  2088,  1012,  1996, 24502,  4164,  2523,
       18067,  3591,  1023,  4113,  3324,  1010,  2184,  6759,  1010,
        1018,  2082,  4616,  1010,  1016,  9656,  1010,  1019,  3541,
       15641,  4616,  1010, 13960, 14767,  2012,  2225,  3028,  2152,
        2082,  1998,  1996, 24502,  3360,  4322,  1010,  4616,  2005,
        1996,  7156,  2188,  1998,  1996,  7939, 11475,  2415,  1010,
        1019,  4616,  2006, 13970,  2319,  2003,  7397,  2444,  1011,
        1999,  2035,  2062,  2084,  2382,  2824,  2023,  2161,  1012,
        2009,  2001,  2178,  6919,  2161,  2013,  2019,  6018,  3233,
        8400,  1998,  2200,  2092,  2363,  1010, 17003,  2256,  5166,
       21796,  2005,  2119, 15002,  2015,  1998,  5779,  1012,  1999,
        2035,  1010]

In [91]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [93]:
# # Use train_test_split to split our data into train and validation sets for training
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.2)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.2)

# Use train_test_split to split our data into train and validation sets for training
validation_inputs, validation_labels = [input_ids, labels]
validation_masks = attention_masks

In [97]:
len(validation_inputs), len(validation_labels), len(validation_masks)

(38607, 38607, 38607)

In [98]:
# Convert all of our data into torch tensors, the required datatype for our model
# train_inputs = torch.tensor(train_inputs)
# train_labels = torch.tensor(train_labels)
# train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [99]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [78]:
train_data[5], len(train_data)

((tensor([  101, 15775, 24860,  4212,  4547,  3192,  2000,  5770,  1996, 15775,
          24860,  4212, 11819,  2098,  2082,  2212,  2000,  3073,  2490,  2005,
           3454,  1998,  3450,  1999,  1996, 15775, 24860,  4212, 11819,  2098,
           2082,  2212,  3946,  2000,  1996, 15775, 24860,  4212,  3502,  2604,
           1002,  3486, 17914,  1998,  1002,  1015,  2063,  8889,  2692,  2000,
          15775, 24860,  4212,  5981,  2252,  1025,  5857, 11095,  2000, 15775,
          24860,  4212, 11819,  2098,  2082,  2212,  2005,  1996,  2311,  1997,
           1996,  8144,  2415,  2006,  1996, 15775, 24860,  4212,  2152,  2082,
           3200,  1025,  3946,  2000, 12785,  2495,  3836,  6566,  2005, 12785,
           2493,   102,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

In [12]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=len(df_UCF_train.NTEE1.unique()))
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace

In [13]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [14]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [15]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [29]:
t = [] 
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    
    
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.6066325898500528


Epoch:  25%|██▌       | 1/4 [1:06:23<3:19:10, 3983.57s/it]

Validation Accuracy: 0.8648744824016563
Train loss: 0.4018097531385247


Epoch:  50%|█████     | 2/4 [2:13:01<2:12:55, 3987.80s/it]

Validation Accuracy: 0.8520639233954451
Train loss: 0.3226928493942476


Epoch:  75%|███████▌  | 3/4 [3:19:55<1:06:35, 3995.87s/it]

Validation Accuracy: 0.8716679606625258
Train loss: 0.24706817368357578


Epoch: 100%|██████████| 4/4 [4:26:12<00:00, 3993.19s/it]  

Validation Accuracy: 0.8677536231884058


In [100]:
# Validation

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
logits_all=[]
label_ids_all=[]

# Evaluate data for one epoch
for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    logits_all+=list(np.argmax(logits, axis=1))
    label_ids = b_labels.to('cpu').numpy()
    label_ids_all+=list(label_ids)

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Validation Accuracy: 0.8693264982049158


In [101]:
logits_all_letter=preprocessing.LabelEncoder().fit(df_UCF_train.NTEE1.values).inverse_transform(logits_all)
label_ids_all_letter=preprocessing.LabelEncoder().fit(df_UCF_train.NTEE1.values).inverse_transform(label_ids_all)

In [102]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_true=label_ids_all_letter, y_pred=logits_all_letter, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

          A     0.9468    0.8835    0.9938    0.9140    0.9370    0.8683      4291
          B     0.9114    0.9057    0.9824    0.9086    0.9433    0.8830      6419
          C     0.7935    0.8597    0.9951    0.8253    0.9249    0.8439       827
          D     0.8873    0.9516    0.9967    0.9183    0.9739    0.9442      1034
          E     0.8858    0.8240    0.9933    0.8538    0.9047    0.8046      2307
          F     0.7423    0.7532    0.9963    0.7477    0.8663    0.7322       543
          G     0.8259    0.8381    0.9936    0.8320    0.9126    0.8198      1353
          H     0.4775    0.6746    0.9976    0.5592    0.8203    0.6512       126
          I     0.8268    0.8514    0.9965    0.8389    0.9211    0.8361       740
          J     0.8495    0.8224    0.9956    0.8357    0.9049    0.8046      1132
          K     0.8340    0.8276    0.9977    0.8308    0.9087    0.8117       522
   

In [104]:
Counter(logits_all_letter)

Counter({'P': 2105,
         'A': 4004,
         'N': 3988,
         'Y': 1672,
         'G': 1373,
         'E': 2146,
         'C': 896,
         'S': 3776,
         'I': 762,
         'B': 6379,
         'M': 1215,
         'D': 1109,
         'L': 1566,
         'X': 1158,
         'Q': 556,
         'K': 518,
         'R': 224,
         'O': 377,
         'U': 268,
         'W': 2120,
         'H': 178,
         'J': 1096,
         'F': 551,
         'T': 481,
         'V': 89})

In [107]:
t=pd.DataFrame([logits_all_letter, label_ids_all_letter]).T.rename(columns={0:'pred', 1:'true'})

In [111]:
len(t[t.pred==t.true])/len(t)

0.8692983137772943

In [113]:
t.sample(50)

,pred,true
1937,N,N
37047,G,G
10310,N,N
3989,S,S
4727,N,N
13050,L,L
20474,A,A
21304,B,B
21964,B,B
3206,Y,Y
